In [2]:
from datasets import load_from_disk
dataset = load_from_disk("../data/train_dataset/")
print(dataset)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})


반각 문자 변환

In [71]:
import re
import unicodedata
from datasets import DatasetDict

def normalize_context(
          dataset: DatasetDict
) -> DatasetDict:
     '''
     dataset의 context를 반각 문자로 변환
     '''
     def normalize(example):
          return {"context": unicodedata.normalize('NFKC', example['context'])}
     return normalize(dataset)

In [18]:
def normalize_answer(
        dataset: DatasetDict
) -> DatasetDict:
      '''
      dataset의 answer를 반각 문자로 변환
      '''
      def normalize(example):
        example['answers']['text'][0]=unicodedata.normalize('NFKC', example['answers']['text'][0])
        return {"answers": example['answers']}
      return normalize(dataset)

In [19]:
def normalize_question(
          dataset: DatasetDict
) -> DatasetDict:
     '''
     dataset의 question을 반각 문자로 변환
     '''
     def normalize(example):
        return {"question": unicodedata.normalize('NFKC', example['question'])}
     return normalize(dataset)

특수문자 제거

In [72]:
def sub_text(
          text: str
) -> str:
        cp = re.compile('\\\\n|\*|\\n|\\|#')
        
        text = re.sub(r"[“”‘’]", "\'", text)
        text = re.sub(r"[〈<＜「≪《『]", "<", text)
        text = re.sub(r"[〉>＞」≫》』]", ">", text)
        text = cp.sub('', text)
        return text

In [29]:
def dataset_sub_context(
        dataset: DatasetDict
) -> DatasetDict:
    '''
    dataset 내의 context에 포함되어 있는 특수문자 제거 
    '''
    def sub(example):
        answer_start = example['answers']['answer_start'][0]
        new_answer_start = len(sub_text(example['context'][:answer_start]))
        example['answers']['answer_start'][0] = new_answer_start
        return {"context": sub_text(example['context']), "answers": example['answers']}
    return sub(dataset)

In [73]:
preprocessed_dataset = dataset.map(normalize_context).map(normalize_answer).map(normalize_question).map(dataset_sub_context)

Map: 100%|██████████| 240/240 [00:00<00:00, 4692.18 examples/s]


기존 경로에 데이터셋 저장

In [74]:
preprocessed_dataset.save_to_disk("../data/train_dataset/")

Saving the dataset (1/1 shards): 100%|██████████| 240/240 [00:00<00:00, 34139.35 examples/s]
